In [15]:
import pandas as pd
import os
import datetime
import holidays

In [16]:
file_path = os.path.join('./contract_3_5_month_minutes' + os.path.sep)

In [367]:

_TIME_DELTA_15_MINUTES = datetime.timedelta(minutes=15)

_TW_HOLIDAYS_OBJ = holidays.country_holidays('TW')

_TW_HOLIDAYS_YMD = set([
    (2022, 1, 31),
    (2022, 2, 1),
    (2022, 2, 4),
    (2022, 2, 5),
    (2022, 4, 5),
    (2022, 5, 1),
    (2022, 9, 10),
    (2023, 1, 2),
    (2023, 1, 25),
    (2023, 1, 26),
    (2023, 4, 5),
    (2023, 5, 1),
])

_TW_NON_HOLIDAYS_YMD = set([
    (2022, 10, 11),
    (2022, 1, 1)
])

HD_HALF_BUSY = 0 # 半尖峰
HD_BUSY = 1 # 尖峰
# HD_SAT_HALF_BUSY = 2 # 週六半尖峰
HD_NOT_BUSY = 2  # 離峰


HD_BUSY_STRS = ["半尖峰", "尖峰", "週六半尖峰", "離峰"]


def is_holiday(dt):
    """
    Return True if |dt| is a holiday, where dt can be a string, datetime obj, or date obj.

    Taipower's 國定假日(離峰日)定義:
    開國紀念日  1/1
    春節        農曆除夕~1/5
    和平紀念日  2/28
    兒童節      4/4
    民族掃墓節  4/4或4/5
    勞動節      5/1
    端午節      農曆5/5
    中秋節      農曆8/15
    國慶日      10/10

    Note:
    補假日不算離峰日, 如2022/9/10(六)是中秋節，9/9(五)補假, 9/9仍算成平日。
    同理，補上班日仍算離峰日, 如2023/1/7（六）是上班日，仍算離峰日。
    """
    assert isinstance(dt, datetime.datetime), f"expect datetime type, got {type(dt)}"
    ymd = (dt.year, dt.month, dt.day)
    if ymd in _TW_HOLIDAYS_YMD:
        return True
    if ymd in _TW_NON_HOLIDAYS_YMD:
        return False
    return dt in _TW_HOLIDAYS_OBJ

def get_time_slot_weekday(dt):
    dt = dt - _TIME_DELTA_15_MINUTES
    if is_holiday(dt):
        return 6
    return dt.weekday()

def label_type(row):
    """
    TYPE:
        假日半尖峰 (六)(夏季)09:00~24:00 (非夏季)06:00~11:00、14:00~24:00                -> 0
        假日離峰   (六)(夏季)00:00~09:00 (非夏季)00:00~06:00、11:00~14:00 (日)0:00~24:00 -> 1
        平日尖峰   (夏季)16:00~22:00                                                    -> 2
        平日半尖峰 (夏季)09:00~16:00、22:00~24:00 (非夏季)06:00~11:00、14:00~24:00       -> 3
        平日離峰   (夏季)00:00~09:00              (非夏季)00:00~06:00、11:00~14:00       -> 4
        day_of_week 0 表示星期一、1 表示星期二、...、6 表示星期日
    """
    day_of_week = get_time_slot_weekday(row)
    row = row - _TIME_DELTA_15_MINUTES
    month = int(row.strftime('%m'))
    # day_of_week = int(row.weekday())
    hour = int(row.strftime('%H'))

    # 週六
    if day_of_week == 5:
        # 夏月
        if (month>=6) and (month<=9):
            # 0900~2400 假日半尖峰
            if (hour >= 9) and (hour <= 24):
                return  0
            # 0000~0900 假日離峰 (日)0000~2400
            else:
                return  1
        # 非夏月
        else:
            # 0600~1100 or 1400~2400
            if ((hour >= 6) and (hour <= 11)) or ((hour >= 14) and (hour <= 24)):
                return  HD_HALF_BUSY
            # 0000~0900 假日離峰 (日)0000~2400
            else:
                return  HD_NOT_BUSY

    #週日 國定假日
    elif day_of_week == 6:
        return HD_NOT_BUSY

    # 平日
    else:
        # summer 6~9
        if (month>=6) and (month<=9):
            if (hour >= 16) and (hour <= 22):
                return  2
            elif ((hour >= 9) and (hour <= 16)) or ((hour >= 22) and (hour <= 24)):
                return  3
            else:
                return  4
        # 非夏月 平日半尖峰
        else:
            if ((hour >= 6) and (hour <= 11)) or ((hour >= 14) and (hour <= 24)):
                return  HD_HALF_BUSY
            # 平日離峰
            else:
                return  HD_NOT_BUSY


def plant_analysis(df_plant):
    df_plant_normal = df_plant.copy()
    df_col = df_plant.columns[1:]
    ### 為原本的consumption table標上時段標籤
    df_plant['時間'] = df_plant['時間'] - _TIME_DELTA_15_MINUTES
    df_plant_normal['type'] = list(df_plant['時間'].apply(label_type))
    agg_dict = {c : 'sum' for c in df_col}
    df_plant_normal = df_plant_normal.groupby(['type']).agg(agg_dict)
    return df_plant_normal

def client_analysis(df_client):
    df_client_normal = df_client.copy()
    df_col = df_client.columns[1:]
    ### 為原本的consumption table標上時段標籤
    df_client['時間'] = df_client['時間'] - _TIME_DELTA_15_MINUTES
    df_client_normal['type'] = list(df_client['時間'].apply(label_type))
    agg_dict = {c : 'sum' for c in df_col}
    df_consumption_normal = df_client_normal.groupby(['type']).agg(agg_dict)
    return df_consumption_normal

def distribute_power(df_p, df_c):
    print(df_p['type'])

In [364]:
pl_g = pd.read_csv(file_path + 'power_generation.csv', parse_dates=['時間'])
pl_g = plant_analysis(pl_g)
pl_g.to_csv('power_generation.csv', encoding="utf-8-sig")

In [365]:
pl_c = pd.read_csv(file_path + 'power_consumption.csv', parse_dates=['時間'])
pl_c = client_analysis(pl_c)
pl_c.to_csv('power_consumption.csv', encoding="utf-8-sig")

In [582]:
pl_g = pd.read_csv('power_generation.csv')
cl_c = pd.read_csv('power_consumption.csv')
print(pl_g, pl_c)

pl_info = pd.read_csv(file_path + 'plant_information.csv')
cl_info = pd.read_csv(file_path + 'client_information.csv')

pl_info = pl_info.sort_values(by='FIT')
cl_info = cl_info.sort_values(by='綠電售價', ascending=False)
print(cl_info)

   type   電廠001   電廠002
0     0  447200  559000
1     2  342680  428350       64-67-0001-13-1  64-67-0002-13-1  64-67-0003-13-1  64-67-0004-13-1  \
type                                                                       
0              379638         455565.6         303710.4           560350   
2              180670         216804.0         144536.0           481250   

      64-67-0005-13-1  
type                   
0           2186722.5  
2           1040683.5  
      用電端            用電端電號  月上限  年上限  RE  最低年轉供      綠電售價
1  用戶0002  64-67-0002-13-1  NaN  NaN   1    0.0  6.225529
3  用戶0004  64-67-0004-13-1  NaN  NaN   1    0.0  6.053718
2  用戶0003  64-67-0003-13-1  NaN  NaN   1    0.0  5.957455
4  用戶0005  64-67-0005-13-1  NaN  NaN   1    0.0  5.582150
0  用戶0001  64-67-0001-13-1  NaN  NaN   1    0.0  5.537592


In [614]:
# first 電廠
not_enough = 0
ret = 0
cl_distribute = []
pl_distribute = []
for i in pl_info['發電端']:
    print(f"總供電: {pl_g[i][0:1].to_string(index=False)}")
    pl_distribute.append(i)
    for j in cl_info['用電端電號']:
        print("分配:",j)
        cl_distribute.append(j)
        ret = pl_g[i][0:1] - cl_c[j][0:1]
        if float(ret.to_string(index=False)) < 0:
            not_enough = ret
            print("不夠分:",not_enough)
            print(cl_distribute)
            break
    break

cl_info_new = cl_info.用電端電號
pl_info_new = pl_info.發電端

for j in pl_info_new:
    if j not in pl_distribute:
        now_v = pl_g[j][0:1] + not_enough
        print(now_v)

for i in cl_info_new:
    if i not in cl_distribute:
        pass
                # if float(ret.to_string(index=False)) < 0:
                #     break

            # print(list(cl_info['用電端電號']).remove(distribute))
            # break
        # print(set(cl_info['用電端電號'], distribute))


總供電: 559000
分配: 64-67-0002-13-1
分配: 64-67-0004-13-1
不夠分: 0   -1350
dtype: int64
['64-67-0002-13-1', '64-67-0004-13-1']
0    445850
dtype: int64
